In [1]:
import re, csv, time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, UnexpectedAlertPresentException, NoAlertPresentException, ElementClickInterceptedException 
from selenium.webdriver.chrome.options import Options

# ────────────────────── 0. 환경 설정
CHROME_PATH = r"C:\Users\tksmd\chromedriver-win64\chromedriver.exe"

opt = Options()
opt.add_argument("--disable-blink-features=AutomationControlled")   # WebDriver 플래그 숨김
opt.add_experimental_option("excludeSwitches", ["enable-automation"])

BASE        = "https://www.seniorro.or.kr:4431"
URL_SEARCH  = f"{BASE}/noin/searchJob.do"

DISTRICTS = [  # 전국 광역자치단체
    "서울특별시 강남구","서울특별시 강동구","서울특별시 강북구","서울특별시 강서구",
    "서울특별시 관악구","서울특별시 광진구","서울특별시 구로구","서울특별시 금천구",
    "서울특별시 노원구","서울특별시 도봉구","서울특별시 동대문구","서울특별시 동작구",
    "서울특별시 마포구","서울특별시 서대문구","서울특별시 서초구","서울특별시 성동구",
    "서울특별시 성북구","서울특별시 송파구","서울특별시 양천구","서울특별시 영등포구",
    "서울특별시 용산구","서울특별시 은평구","서울특별시 종로구","서울특별시 중구",
    "서울특별시 중랑구",
    "부산광역시 강서구", "부산광역시 금정구", "부산광역시 기장군", "부산광역시 남구",
    "부산광역시 동구", "부산광역시 동래구", "부산광역시 부산진구", "부산광역시 북구",
    "부산광역시 사상구", "부산광역시 사하구", "부산광역시 서구", "부산광역시 수영구",
    "부산광역시 연제구", "부산광역시 영도구", "부산광역시 중구", "부산광역시 해운대구",
    "인천광역시 강화군", "인천광역시 계양구", "인천광역시 남동구",
    "인천광역시 동구", "인천광역시 미추홀구", "인천광역시 부평구",
    "인천광역시 서구", "인천광역시 연수구", "인천광역시 옹진군",
    "인천광역시 중구",
    "대구광역시 남구", "대구광역시 달서구", "대구광역시 달성군",
    "대구광역시 동구", "대구광역시 북구", "대구광역시 서구",
    "대구광역시 수성구", "대구광역시 중구",
    "광주광역시 광산구", "광주광역시 남구", "광주광역시 동구",
    "광주광역시 북구", "광주광역시 서구",
    "대전광역시 대덕구", "대전광역시 동구", "대전광역시 서구",
    "대전광역시 유성구", "대전광역시 중구",
    "울산광역시 남구", "울산광역시 동구", "울산광역시 북구",
    "울산광역시 울주군", "울산광역시 중구",
    "세종특별자치시 세종시",
    "경기도 가평군", "경기도 고양시", "경기도 고양시 덕양구", "경기도 고양시 일산동구",
    "경기도 고양시 일산서구", "경기도 과천시", "경기도 광명시", "경기도 광주시",
    "경기도 구리시", "경기도 군포시", "경기도 김포시", "경기도 남양주시",
    "경기도 동두천시", "경기도 부천시", "경기도 성남시", "경기도 성남시 분당구",
    "경기도 성남시 수정구", "경기도 성남시 중원구", "경기도 수원시",
    "경기도 수원시 권선구", "경기도 수원시 영통구", "경기도 수원시 장안구",
    "경기도 수원시 팔달구", "경기도 시흥시", "경기도 안산시",
    "경기도 안산시 단원구", "경기도 안산시 상록구", "경기도 안성시",
    "경기도 안양시", "경기도 안양시 동안구", "경기도 안양시 만안구",
    "경기도 양주시", "경기도 양평군", "경기도 여주시", "경기도 연천군",
    "경기도 오산시", "경기도 용인시", "경기도 용인시 기흥구",
    "경기도 용인시 수지구", "경기도 용인시 처인구", "경기도 의왕시",
    "경기도 의정부시", "경기도 이천시", "경기도 파주시", "경기도 평택시",
    "경기도 포천시", "경기도 하남시", "경기도 화성시",
    "충청남도 계룡시", "충청남도 공주시", "충청남도 금산군", "충청남도 논산시",
    "충청남도 당진시", "충청남도 보령시", "충청남도 부여군", "충청남도 서산시",
    "충청남도 서천군", "충청남도 아산시", "충청남도 예산군", "충청남도 천안시",
    "충청남도 천안시 동남구", "충청남도 천안시 서북구", "충청남도 청양군",
    "충청남도 태안군", "충청남도 홍성군",
    "충청북도 괴산군", "충청북도 단양군", "충청북도 보은군", "충청북도 영동군",
    "충청북도 옥천군", "충청북도 음성군", "충청북도 제천시", "충청북도 증평군",
    "충청북도 진천군", "충청북도 청주시", "충청북도 청주시 상당구",
    "충청북도 청주시 서원구", "충청북도 청주시 청원구",
    "충청북도 청주시 흥덕구", "충청북도 충주시","충청남도 계룡시", "충청남도 공주시", "충청남도 금산군", "충청남도 논산시",
    "전라남도 강진군", "전라남도 고흥군", "전라남도 곡성군", "전라남도 광양시",
    "전라남도 구례군", "전라남도 나주시", "전라남도 담양군", "전라남도 목포시",
    "전라남도 무안군", "전라남도 보성군", "전라남도 순천시", "전라남도 신안군",
    "전라남도 여수시", "전라남도 영광군", "전라남도 영암군", "전라남도 완도군",
    "전라남도 장성군", "전라남도 장흥군", "전라남도 진도군", "전라남도 함평군",
    "전라남도 해남군", "전라남도 화순군",
    "전라북도 고창군", "전라북도 군산시", "전라북도 김제시", "전라북도 남원시",
    "전라북도 무주군", "전라북도 부안군", "전라북도 순창군", "전라북도 완주군",
    "전라북도 익산시", "전라북도 임실군", "전라북도 장수군", "전라북도 전주시",
    "전라북도 전주시 덕진구", "전라북도 전주시 완산구", "전라북도 정읍시",
    "전라북도 진안군",
    "경상남도 거제시", "경상남도 거창군", "경상남도 고성군", "경상남도 김해시",
    "경상남도 남해군", "경상남도 밀양시", "경상남도 사천시", "경상남도 산청군",
    "경상남도 양산시", "경상남도 의령군", "경상남도 진주시", "경상남도 창녕군",
    "경상남도 창원시", "경상남도 창원시 마산합포구", "경상남도 창원시 마산회원구",
    "경상남도 창원시 성산구", "경상남도 창원시 의창구", "경상남도 창원시 진해구",
    "경상남도 통영시", "경상남도 하동군", "경상남도 함안군", "경상남도 함양군",
    "경상남도 합천군",
    "경상북도 경산시", "경상북도 경주시", "경상북도 고령군", "경상북도 구미시",
    "경상북도 군위군", "경상북도 김천시", "경상북도 문경시", "경상북도 봉화군",
    "경상북도 상주시", "경상북도 성주군", "경상북도 안동시", "경상북도 영덕군",
    "경상북도 영양군", "경상북도 영주시", "경상북도 영천시", "경상북도 예천군",
    "경상북도 울릉군", "경상북도 울진군", "경상북도 의성군", "경상북도 청도군",
    "경상북도 청송군", "경상북도 칠곡군", "경상북도 포항시",
    "경상북도 포항시 남구", "경상북도 포항시 북구",
    "강원도 강릉시", "강원도 고성군", "강원도 동해시", "강원도 삼척시",
    "강원도 속초시", "강원도 양구군", "강원도 양양군", "강원도 영월군",
    "강원도 원주시", "강원도 인제군", "강원도 정선군", "강원도 철원군",
    "강원도 춘천시", "강원도 태백시", "강원도 평창군", "강원도 홍천군",
    "강원도 화천군", "강원도 횡성군",
    "제주특별자치도 서귀포시", "제주특별자치도 제주시"
]

# 16개 고정 컬럼 이름
HEADERS = [
    "기관", "사업체명", "기관소재지", "직무요약",
    "사업유형", "계획인원", "참여인원", "모집기간",
    "근무지역(상세)", "직무내용", "모집인원(상세)",
    "주근무시간", "월근무시간", "임금액",
    "구인담당자", "연락처"
]

# ────────────────────── 1. 셀레니엄 준비
drv = webdriver.Chrome(service=Service(CHROME_PATH), options=opt)
wait = WebDriverWait(drv, 10)
drv.get(URL_SEARCH)

# ─────────── 1.5 팝업 닫기
def dismiss_security_popup(driver):
    """보안 탐지 알림이 떠 있으면 닫고 True, 없으면 False"""
    try:
        alert = driver.switch_to.alert
        alert.accept()
        time.sleep(1)          # 페이지가 다시 로드될 시간을 짧게 줌
        return True
    except NoAlertPresentException:
        return False

# ────────────────────── 2. 목록 한 <li> 8 항목 파싱
def parse_list_li(li):
    # ① 기관 / 사업체명 / 근무지
    region_txt = li.find_element(By.CSS_SELECTOR, ".info-stit .region").text.strip()
    #   '서울강서시니어클럽  │ 사업체명 : 지축역...' 형태
    org   = region_txt.split("│")[0].strip()
    comp  = re.search(r"사업체명\s*:\s*(.+)", region_txt)
    comp  = comp.group(1).strip() if comp else ""

    loc   = li.find_element(By.CSS_SELECTOR,
                            ".info-stit span:nth-of-type(2)").text.strip()

    # ② 직무요약(검은 굵은 글씨)
    summary = li.find_element(By.CSS_SELECTOR, ".info-tit strong").text.replace("\n"," ").strip()

    # ③ 모집기간
    period  = li.find_element(By.CSS_SELECTOR, ".info-proc .info-date").text.strip()

    # ④ 사업유형(아이콘 옆에 두 줄)
    biztype = li.find_element(By.CSS_SELECTOR, ".icon-box strong").text.replace("\n"," ").strip()

    # ⑤ 계획/참여 인원
    ppl_txt = li.find_element(By.CSS_SELECTOR, ".icon-info").text
    plan    = re.search(r"계획인원\s*([\d,]+)", ppl_txt)
    join    = re.search(r"참여인원\s*([\d,]+)", ppl_txt)
    plan = plan.group(1).replace(",","") if plan else ""
    join = join.group(1).replace(",","") if join else ""

    # ⑥ 상세 페이지 URL & <a> 텍스트
    a_tag = li.find_element(By.TAG_NAME, "a")
    href  = a_tag.get_attribute("href")
    url   = href if href.startswith("http") else BASE + href
    title_txt = a_tag.text.strip()

    row = {
        "기관":org, "사업체명":comp, "기관소재지":loc,
        "직무요약":summary, "사업유형":biztype,
        "계획인원":plan, "참여인원":join, "모집기간":period,
        "_url":url, "_title":title_txt   # 상세 페이지용 임시 필드
    }
    return row


# ────────────────────── 3. 상세 페이지 8 항목 파싱
def parse_detail(url):
    drv.get(url)

    def t(id_): return drv.find_element(By.ID, id_).text.strip()

    return {
        "근무지역(상세)" : t("dDtlWorkArea"),
        "직무내용"      : t("dDetCnts").replace("\n"," ").strip(),
        "모집인원(상세)" : t("dWokerCnt"),
        "주근무시간"    : f"{t('dWdayPweek')}일/{t('dWtimePday')}시간",
        "월근무시간"    : t("dWdWtimePmon"),
        "임금액"        : t("dPayMon"),
        "구인담당자"    : t("dClerk"),
        "연락처"        : t("dClerkContt")
    }


# ────────────────────── 4. 구 단위 스크래핑
def scrape_gu(region):
    box = wait.until(EC.presence_of_element_located((By.ID, "seachKeyword")))
    keyword = region.split()[0][:2]        # 예) '부산광역시...' → '부산'
    box.clear()
    box.send_keys(keyword)
    time.sleep(0.8)
    for s in drv.find_elements(By.CSS_SELECTOR,"#searchSimilar span"):
        if s.text.strip() == region:
            try:
                s.click()                           # ① 시도
            except ElementClickInterceptedException:
                drv.execute_script("arguments[0].click()", s)  # ② JS 강제 클릭
            break

    # li 가 렌더링될 때까지 대기
    try:
        dismiss_security_popup(drv)
        wait.until(lambda d: len(d.find_elements(
            By.CSS_SELECTOR, "#rsList02>li")) > 0)
    except TimeoutException:
        print("   🔹 공고 없음"); return []

    rows = []
    while True:
        li_elems = drv.find_elements(By.CSS_SELECTOR, "#rsList02>li")
        if not li_elems:
            break  # 뒤로 왔는데 목록이 0개면 루프 탈출

        try:
            for li in li_elems:
                rows.append(parse_list_li(li))
            break
        except StaleElementReferenceException:
            continue

    # 상세 페이지 돌며 8개 추가
    for r in rows:
        try:
            detail = parse_detail(r.pop("_url"))
            r.update(detail)
        except Exception as e:
            print(f"   ⚠ 상세 오류: {e}")
        finally:
            drv.back()
            try:
                dismiss_security_popup(drv)
                wait.until(
                    lambda d: len(
                        d.find_elements(By.CSS_SELECTOR, "#rsList02>li")
                    ) >= 0          # 리스트 자체만 나타나면 통과 (0 건도 허용)
                )
            except TimeoutException:
                pass                # 결과가 0 건이라도 넘어가도록 그냥 통과

    return rows


# ────────────────────── 5. 전체 루프
results=[]
for i,gu in enumerate(DISTRICTS,1):
    print(f"[{i}/{len(DISTRICTS)}] {gu}")
    results.extend(scrape_gu(gu))
    time.sleep(0.5)

drv.quit()

# ────────────────────── 6. CSV 저장
if results:
    # 임시 키 제거 및 누락 컬럼 채우기
    for r in results:
        r.pop("_title", None)
    for r in results:
        for k in HEADERS: r.setdefault(k,"")

    with open("korea_jobs_fixed16_real_final.csv","w",newline="",encoding="cp949") as f:
        w=csv.DictWriter(f,fieldnames=HEADERS)
        w.writeheader(); w.writerows(results)

    print(f"\n✅ {len(results)}건 → korea_jobs_fixed16_real_final.csv 저장 완료")
else:
    print("수집 결과가 없습니다.")


[1/265] 서울특별시 강남구
[2/265] 서울특별시 강동구
[3/265] 서울특별시 강북구
   🔹 공고 없음
[4/265] 서울특별시 강서구
[5/265] 서울특별시 관악구
[6/265] 서울특별시 광진구
   🔹 공고 없음
[7/265] 서울특별시 구로구
[8/265] 서울특별시 금천구
[9/265] 서울특별시 노원구
[10/265] 서울특별시 도봉구
[11/265] 서울특별시 동대문구
   🔹 공고 없음
[12/265] 서울특별시 동작구
   🔹 공고 없음
[13/265] 서울특별시 마포구
[14/265] 서울특별시 서대문구
   🔹 공고 없음
[15/265] 서울특별시 서초구
   🔹 공고 없음
[16/265] 서울특별시 성동구
[17/265] 서울특별시 성북구
[18/265] 서울특별시 송파구
[19/265] 서울특별시 양천구
[20/265] 서울특별시 영등포구
[21/265] 서울특별시 용산구
[22/265] 서울특별시 은평구
   🔹 공고 없음
[23/265] 서울특별시 종로구
[24/265] 서울특별시 중구
[25/265] 서울특별시 중랑구
[26/265] 부산광역시 강서구
[27/265] 부산광역시 금정구
[28/265] 부산광역시 기장군
   🔹 공고 없음
[29/265] 부산광역시 남구
[30/265] 부산광역시 동구
[31/265] 부산광역시 동래구
[32/265] 부산광역시 부산진구
[33/265] 부산광역시 북구
   🔹 공고 없음
[34/265] 부산광역시 사상구
   🔹 공고 없음
[35/265] 부산광역시 사하구
   🔹 공고 없음
[36/265] 부산광역시 서구
[37/265] 부산광역시 수영구
[38/265] 부산광역시 연제구
[39/265] 부산광역시 영도구
[40/265] 부산광역시 중구
   🔹 공고 없음
[41/265] 부산광역시 해운대구
   🔹 공고 없음
[42/265] 인천광역시 강화군
   🔹 공고 없음
[43/265] 인천광역시 계양구
[44/265] 인천광역시 남동구
[45/265] 인천광역시 동구
[4